Notebook Entrainement de modèle depuis les données mongoDB vers MLFLOW

In [1]:
import os
import json
import pymongo
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from pyspark.sql.types import StructType, StructField, StringType, MapType, TimestampType
from transformers import DistilBertTokenizer
from transformers import DistilBertModel
import torch
import torch.nn as nn

/opt/bitnami/python/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

# Define a UDF to convert the string representation of the array to an actual array of floats
def parse_array(col):
    try:
        return [float(x) for x in col.strip('[]').split(', ')]
    except ValueError:
        return [0.0]*768

parse_array_udf = udf(parse_array, ArrayType(FloatType()))


# Create a Spark session with the MongoDB Spark Connector package
spark = SparkSession.builder \
    .appName("myApp") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()


schema = StructType([
    StructField("user", StringType(), True),
    StructField("repo", StringType(), True),
    StructField("mainLanguage", StringType(), True),
    StructField("languages", MapType(StringType(), StringType()), True),
    StructField("readme", StringType(), True),
    StructField("processed_readme", StringType(), True),
    StructField("last_updated", TimestampType(), True),
])

df = spark.read \
    .format("mongo") \
    .option("database", "dev") \
    .option("collection", "raw_data") \
    .option("uri", "mongodb://mongo:27017/") \
    .schema(schema) \
    .load()

df = df.withColumn("processed_readme", parse_array_udf(df["processed_readme"]))

df.show()

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4f8a407c-3457-45c6-a362-684ae1105163;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 158ms :: artifacts dl 11ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       

+-----------------+--------------------+------------+--------------------+--------------------------+--------------------+--------------------+
|             user|                repo|mainLanguage|           languages|                    readme|    processed_readme|        last_updated|
+-----------------+--------------------+------------+--------------------+--------------------------+--------------------+--------------------+
|               d3|                  d3|       Shell|{Shell -> NULL, J...|      # D3: Data-Driven...|[-0.47322002, -0....|2025-03-09 12:24:...|
|   papers-we-love|      papers-we-love|       Shell|     {Shell -> NULL}|      ﻿## ![Papers We L...|[-0.2555281, -0.2...|2025-03-09 12:25:...|
|           nvm-sh|                 nvm|       Shell|{Dockerfile -> NU...|      <a href="https://...|[-0.34048513, -0....|2025-03-09 12:26:...|
|             base|                node|       Shell|{Dockerfile -> NU...|      ![Base](logo.webp...|[-0.31638318, -0....|2025-03-09 12:

In [3]:
num_lines = df.count()
print(f"Number of lines in the dataframe: {num_lines}")

Number of lines in the dataframe: 7026


In [4]:
# Charger les données depuis MongoDB
def load_data_from_mongo():
    data = df.select("processed_readme", "mainLanguage").collect()
    texts = [row["processed_readme"] for row in data]
    labels = [row["mainLanguage"] for row in data]
    return torch.tensor(texts), labels


In [5]:
# Charger le tokenizer et le modèle DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
bert_model = DistilBertModel.from_pretrained("distilbert-base-uncased")


# Charger et préparer les données
texts, labels = load_data_from_mongo()
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)



In [6]:
# Transformer les textes en embeddings DistilBERT
#document_vectors = encode_texts_with_bert(texts, tokenizer, bert_model)

# Division des données en train/test
#X_train, X_test, y_train, y_test = train_test_split(document_vectors, encoded_labels, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(texts, encoded_labels, test_size=0.2, random_state=42)


In [7]:
#Entraînement du classifieur
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [8]:
#Évaluation du modèle
train_score = classifier.score(X_train, y_train)
test_score = classifier.score(X_test, y_test)

print(f"Train Accuracy: {train_score:.4f}")
print(f"Test Accuracy: {test_score:.4f}")



Train Accuracy: 0.9998
Test Accuracy: 0.3698


In [9]:
# Enregistrement dans MLflow
mlflow.set_tracking_uri("http://mlflow:8080")

with mlflow.start_run():
    mlflow.sklearn.log_model(classifier, "random_forest_model")
    mlflow.log_param("model_type", "RandomForestClassifier")
    mlflow.log_metric("train_accuracy", train_score)
    mlflow.log_metric("test_accuracy", test_score)
    
    # Save the label mapping to a file and log it as an artifact
    with open("label_mapping.json", "w") as f:
        json.dump(label_encoder.classes_.tolist(), f)
    mlflow.log_artifact("label_mapping.json")
    os.remove("label_mapping.json")

print("Modèle entraîné et enregistré dans MLflow")

2025/03/09 14:48:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run wistful-sow-513 at: http://mlflow:8080/#/experiments/0/runs/c8c8c5320e9f45299b2e5c622d6d0542
🧪 View experiment at: http://mlflow:8080/#/experiments/0
Modèle entraîné et enregistré dans MLflow
